## Part 5 - Labelisation

Part 1

In [ ]:
# Chargement des librairies nécessaires
import pandas as pd
import spacy

# Chargement de la dataset
dataset = pd.read_csv("dataset_CV.csv", delimiter = ";")

# Instanciation de la bibliothèque fr_core_news_lg de Spacy
nlp = spacy.load("fr_core_news_lg")

# Tokenization des textes de CV
dataset["Tokenization_preparation"] = dataset["CV_text"].apply(lambda x : str(x).replace("\n", " "))
dataset["Tokenization"] = dataset["Tokenization_preparation"].apply(lambda x : nlp(x))
dataset["Number_of_tokens"] = dataset["Tokenization"].apply(lambda x : len(x))

In [ ]:
## Création identification du numéro de CV
df = pd.DataFrame(columns = ['CV_Number'])

## Sélection du texte du CV
doc = dataset.iloc[0,2]
doc = nlp(doc)

df['Sentences_CV'] = 0

## Création d'une liste de phrase
list_sentence = []
for sent in doc.sents :
    list_sentence.append(sent)

## Insertion des phrases et suppression des parenthèses
df['Sentences_CV'] = list_sentence
df['Sentences_CV_clean'] = df['Sentences_CV'].apply(lambda x : str(x).replace("(","").replace(")",""))

df['CV_Number'] = dataset.iloc[0,0]

## Création d'une colonne qui compte le nombre de phrase
df['Sentence_line'] = df.index

## Création d'une colonne qui compte le nombre de tokens dans la phrase
df['Nb_tokens'] = df['Sentences_CV'].apply(lambda x : len(x))

## Création d'une colonne pour connaitre la position en pourcentage de la phrase par rapport au texte entier
df['Nb_tokens'].sum()
df['Nb_tokens'][0]

## Création d'une colonne pour positionner la phrase dans le corpus de texte en pourcentage
df['%texte_lu'] = 0

token_count = 0
for i in range(len(df)):
    token_count += df['Nb_tokens'][i]
    df['%texte_lu'][i] = round(((token_count / df['Nb_tokens'].sum()) * 100),2)

## Création pour connaître le % de texte lu à partir de la fin de ligne
df["%texte_lu_fin_ligne"] = 0

nb_tokens_total = df["Nb_tokens"].sum()

for i in range(len(df)):
    df["%texte_lu_fin_ligne"][i] += (nb_tokens_total - df['Nb_tokens'][i]) / nb_tokens_total

## Création d'une colonne pour savoir si le mot est alphanumerique ou pas
df["Is_alpha"] = 0
for i in range(len(df)):
    doc = df.iloc[i,2]
    doc = nlp(doc)
    list_token_is_alpha = []
    for token in doc:
        list_token_is_alpha.append(token.is_alpha)
    df["Is_alpha"][i] = list_token_is_alpha

## Création de la colonne Grammaire pour connaitre le type de mots de la phrase
df["Grammar"] = 0
i = 0
for i in range(len(df)):
    doc = df.iloc[i,2]
    doc = nlp(doc)
    list_token_pos = []
    for token in doc:
        list_token_pos.append(token.pos_)
        df["Grammar"][i] = list_token_pos

## Création de la colonne Label qui sera notre target. Soit 1 il faut supprimer soit 0 on doit garder
df['Label'] = 0

## Création de la dataset avec les CV concatener
dataset_concat = pd.DataFrame()

dataset_concat = dataset_concat.append(df, ignore_index=True)    

## Création d'un fichier Excel avec toutes les phrases de tous les CV

PATH = "/Users/kinn/Desktop/Projet_BlindCV/data_CV.xls"
dataset_concat.to_excel(excel_writer = PATH, index = True)

dataset_concat.shape
